In [57]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [58]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

fake_or_real_the_impostor_hunt_path = kagglehub.competition_download('fake-or-real-the-impostor-hunt')

print('Data source import complete.')


Data source import complete.


In [59]:
import pandas as pd
import random
import nltk
from nltk.corpus import wordnet
import os
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.optim import AdamW
from sklearn.metrics import accuracy_score
from tqdm.auto import tqdm
import zipfile

In [60]:
import nltk
nltk.download('wordnet', download_dir='/kaggle/working/nltk_data')
nltk.download('omw-1.4', download_dir='/kaggle/working/nltk_data')
nltk.data.path.append('/kaggle/working/nltk_data')

[nltk_data] Downloading package wordnet to
[nltk_data]     /kaggle/working/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /kaggle/working/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [61]:
import os
import pandas as pd

def is_model_saved(output_dir, model_filename='pytorch_model.bin'):
    """Checks if a trained model is saved in the output directory."""
    model_path = os.path.join(output_dir, model_filename)
    return os.path.exists(model_path)

def summarize_test_results(submission_df):
    """Summarizes the test results from the submission DataFrame."""
    print("\n--- Test Summary ---")
    total_articles = len(submission_df)
    real_count = (submission_df['real_text_id'] == 1).sum()
    fake_count = (submission_df['real_text_id'] == 2).sum()
    average_confidence = submission_df['confidence'].mean() # Calculate average confidence
    print(f"Total Articles Processed: {total_articles}")
    print(f"Articles Predicted as REAL: {real_count}")
    print(f"Articles Predicted as FAKE: {fake_count}")
    print(f"Average Confidence: {average_confidence:.4f}") # Display average confidence
    print("--------------------------")
    display(submission_df.head()) # Display the first few rows of the submission

In [62]:
def read_text_file(filepath):
    if os.path.exists(filepath):
        with open(filepath, 'r', encoding='utf-8') as f:
            return f.read()
    return ''

def build_text_csv(train_csv, data_dir, file1_name, file2_name, output_csv):
    df = pd.read_csv(train_csv)
    rows = []
    for _, row in df.iterrows():
        article_id = row['id']
        real_text_id = row['real_text_id']
        article_dir = os.path.join(data_dir, f"article_{str(article_id).zfill(4)}")
        file1_path = os.path.join(article_dir, file1_name)
        file2_path = os.path.join(article_dir, file2_name)
        file1_text = read_text_file(file1_path)
        file2_text = read_text_file(file2_path)
        label = 1 if str(article_id) == str(real_text_id) else 0
        rows.append({
            'id': article_id,
            'file_1_text': file1_text,
            'file_2_text': file2_text,
            'label': label
        })
    out_df = pd.DataFrame(rows)
    out_df.to_csv(output_csv, index=False)
    print(f"Wrote {len(out_df)} rows to {output_csv}")

In [63]:
# Example usage
os.makedirs('/kaggle/working', exist_ok=True) # Create the output directory
build_text_csv(os.path.join(fake_or_real_the_impostor_hunt_path, 'data', 'train.csv'), os.path.join(fake_or_real_the_impostor_hunt_path, 'data', 'train'), 'file_1.txt', 'file_2.txt', '/kaggle/working/train_text.csv')

Wrote 95 rows to /kaggle/working/train_text.csv


In [64]:
def synonym_replacement(sentence, n=2):
    words = sentence.split()
    new_words = words.copy()
    random_word_list = list(set([word for word in words if wordnet.synsets(word)]))
    random.shuffle(random_word_list)
    num_replaced = 0
    for random_word in random_word_list:
        synonyms = wordnet.synsets(random_word)
        if not synonyms:
            continue
        synonym_words = set()
        for syn in synonyms:
            for lemma in syn.lemmas():
                synonym = lemma.name().replace('_', ' ')
                if synonym.lower() != random_word.lower():
                    synonym_words.add(synonym)
        if len(synonym_words) > 0:
            synonym = random.choice(list(synonym_words))
            new_words = [synonym if word == random_word else word for word in new_words]
            num_replaced += 1
        if num_replaced >= n:
            break
    return ' '.join(new_words)

In [65]:
def augment_csv(input_csv, output_csv, text_col1, text_col2, label_col, augment_factor=1):
    df = pd.read_csv(input_csv)
    print(df.columns)
    augmented_rows = []
    for _, row in df.iterrows():
        for _ in range(augment_factor):
            new_row = row.copy()
            new_row[text_col1] = synonym_replacement(str(row[text_col1]))
            new_row[text_col2] = synonym_replacement(str(row[text_col2]))
            new_row[label_col] = row[label_col]    # always copy the label
            augmented_rows.append(new_row)
    aug_df = pd.DataFrame(augmented_rows)
    result = pd.concat([df, aug_df], ignore_index=True)
    result.to_csv(output_csv, index=False)
    print('------------------------CHECK--------------------')
    df_check = pd.read_csv('/kaggle/working/train_augmented.csv')
    print(df_check.columns)  # Ensure 'label' is present
    print(df_check.head())


    print(f"Augmented data saved to {output_csv}. Original: {len(df)}, Augmented: {len(result)}")

# Example usage
#augment_csv('/kaggle/input/fake-or-real-the-impostor-hunt/data/train.csv', '/kaggle/working/train_augmented.csv', 'file_1.txt', 'file_2.txt', 'label', augment_factor=1)

input_csv = '/kaggle/working/train_text.csv'
output_csv = '/kaggle/working/train_augmented.csv'
text_col1 = 'file_1_text'  # Change to your actual column name
text_col2 = 'file_2_text'  # Change to your actual column name
label_col = 'label'        # Change to your actual label column name
augment_factor = 1         # Number of augmented samples per original
augment_csv(input_csv, output_csv, text_col1, text_col2, label_col, augment_factor)

Index(['id', 'file_1_text', 'file_2_text', 'label'], dtype='object')
------------------------CHECK--------------------
Index(['id', 'file_1_text', 'file_2_text', 'label'], dtype='object')
   id                                        file_1_text  \
0   0  The VIRSA (Visible Infrared Survey Telescope A...   
1   1  China\nThe goal of this project involves achie...   
2   2  Scientists can learn about how galaxies form a...   
3   3  China\nThe study suggests that multiple star s...   
4   4  Dinosaur Rex was excited about his new toy set...   

                                         file_2_text  label  
0  The China relay network has released a signifi...      0  
1  The project aims to achieve an accuracy level ...      0  
2  Dinosaur eggshells offer clues about what dino...      0  
3  The importance for understanding how stars evo...      0  
4  Analyzing how fast stars rotate within a galax...      0  
Augmented data saved to /kaggle/working/train_augmented.csv. Original: 95, Augm

In [66]:
class TextPairDataset(torch.utils.data.Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len
    def __len__(self):
        return len(self.data)
    def __getitem__(self, index):
        row = self.data.iloc[index]
        text1 = str(row['file_1_text']) # Ensure text is string
        text2 = str(row['file_2_text']) # Ensure text is string
        label = row['label']
        encoding = self.tokenizer.encode_plus(
            text1, text2, add_special_tokens=True, max_length=self.max_len,
            padding='max_length', truncation=True, return_tensors='pt',
            return_attention_mask=True, return_token_type_ids=True
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'token_type_ids': encoding['token_type_ids'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Load data
max_len = 256
batch_size = 16 # Using batch size from config.py
epochs = 4 # Using epochs from config.py
learning_rate = 3e-5 # Using learning rate from config.py

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
df = pd.read_csv('/kaggle/working/train_augmented.csv')

# Add the code here to fill NaN values in text columns with empty strings
df['file_1_text'] = df['file_1_text'].fillna('')
df['file_2_text'] = df['file_2_text'].fillna('')


print('-------------------------------------------------------------')
print(df.columns)

dataset = TextPairDataset(df, tokenizer, max_len=max_len)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2).to(device)
optimizer = AdamW(model.parameters(), lr=learning_rate) # Use learning_rate variable

# Training loop
# Check if model is already saved
output_dir = '/kaggle/working' # Define output directory
if not is_model_saved(output_dir):
    model.train()
    for epoch in range(epochs): # Loop over epochs
        print(f"Epoch {epoch+1}/{epochs}")
        for batch in tqdm(dataloader, desc='Training'):
            optimizer.zero_grad()
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
else:
    print(f"Model already found in {output_dir}. Skipping training.")

-------------------------------------------------------------
Index(['id', 'file_1_text', 'file_2_text', 'label'], dtype='object')


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/4


Training:   0%|          | 0/12 [00:00<?, ?it/s]

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

...

Training:   0%|          | 0/12 [00:00<?, ?it/s]

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Epoch 4/4


Training:   0%|          | 0/12 [00:00<?, ?it/s]

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

In [67]:
from transformers import AutoTokenizer# Example prediction loop for test set

# Example for bert-base-uncased; change to your model as needed
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

submission_results = []
test_dir = os.path.join(fake_or_real_the_impostor_hunt_path, 'data', 'test')
for article_folder in sorted(os.listdir(test_dir)):
    article_path = os.path.join(test_dir, article_folder)
    file1_path = os.path.join(article_path, 'file_1.txt')
    file2_path = os.path.join(article_path, 'file_2.txt')
    text1 = read_text_file(file1_path)
    text2 = read_text_file(file2_path)
    encoding = tokenizer.encode_plus(
        text1, text2, add_special_tokens=True, max_length=max_len,
        padding='max_length', truncation=True, return_tensors='pt',
        return_attention_mask=True, return_token_type_ids=True
    )
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    token_type_ids = encoding['token_type_ids'].to(device)
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
    logits = outputs.logits
    probs = torch.softmax(logits, dim=1).flatten().cpu().numpy()
    if probs[1] > probs[0]:
        real_text_id = 1
        confidence = probs[1] # Store confidence for the predicted class
    else:
        real_text_id = 2
        confidence = probs[0] # Store confidence for the predicted class
    article_id = int(article_folder.split('_')[-1])
    submission_results.append({'id': article_id, 'real_text_id': real_text_id, 'confidence': confidence}) # Include confidence in results

submission_df = pd.DataFrame(submission_results)
submission_df.to_csv('/kaggle/working/submission.csv', index=False)

# Summarize test results
summarize_test_results(submission_df)


--- Test Summary ---
Total Articles Processed: 1068
Articles Predicted as REAL: 0
Articles Predicted as FAKE: 1068
Average Confidence: 0.9964
--------------------------


,id,real_text_id,confidence
0,0,2,0.996516
1,1,2,0.996542
2,2,2,0.996613
3,3,2,0.996626
4,4,2,0.996554
